In [1]:
import pandas as pd
import glob
import os

import multiprocessing
multiprocessing.cpu_count()

16

In [2]:
stocks = pd.read_csv("all_times.csv")
sorted(stocks.time_id.unique())

[5,
 11,
 16,
 31,
 62,
 72,
 97,
 103,
 109,
 123,
 128,
 146,
 147,
 152,
 157,
 159,
 169,
 207,
 211,
 213,
 218,
 227,
 229,
 232,
 250,
 254,
 256,
 266,
 273,
 289,
 297,
 303,
 309,
 310,
 317,
 319,
 325,
 326,
 335,
 337,
 358,
 371,
 373,
 380,
 381,
 411,
 420,
 424,
 427,
 436,
 438,
 454,
 465,
 468,
 487,
 493,
 506,
 535,
 536,
 539,
 545,
 554,
 568,
 579,
 589,
 591,
 595,
 611,
 618,
 627,
 633,
 640,
 646,
 650,
 652,
 659,
 675,
 697,
 709,
 725,
 728,
 735,
 745,
 746,
 748,
 753,
 785,
 817,
 840,
 841,
 852,
 854,
 868,
 886,
 895,
 904,
 908,
 947,
 951,
 957,
 969,
 985,
 991,
 1000,
 1011,
 1016,
 1020,
 1022,
 1028,
 1037,
 1040,
 1057,
 1063,
 1070,
 1075,
 1100,
 1107,
 1119,
 1125,
 1128,
 1135,
 1142,
 1149,
 1158,
 1161,
 1173,
 1176,
 1178,
 1191,
 1198,
 1205,
 1209,
 1213,
 1215,
 1219,
 1227,
 1235,
 1239,
 1242,
 1247,
 1249,
 1255,
 1264,
 1269,
 1273,
 1274,
 1280,
 1289,
 1291,
 1292,
 1302,
 1310,
 1321,
 1322,
 1326,
 1348,
 1350,
 1359,
 1363

In [ ]:
def compute_beta(d):
    c = (((d['wap'] - d['wap_mean']) * (d['market_mean_seconds'] - d['market_mean'])) / len(d)).sum()
    v = (((d['market_mean_seconds'] - d['market_mean']) ** 2) / len(d)).sum()
    d['beta'] = c / v
    return d

def extract_features(files, time_id=5):
    stocks = pd.DataFrame(
        {"stock_id": [], "wap": [], "dom": [], "spread": [], "seconds": [], "time_id": []}
    )

    for i, f in enumerate(files):
        df = pd.read_csv(f)
        
        df = df[df['time_id'] == time_id]

        df["wap"] = (df["bid_price1"] * df["ask_size1"] + df["ask_price1"] * df["bid_size1"]) \
                    / (df["bid_size1"] + df["ask_size1"])

        df["dom"] = (
            df['bid_price1'] * df['bid_size1']
            + df['bid_price2'] * df['bid_size2']
            + df['ask_price1'] * df['ask_size1']
            + df['ask_price2'] * df['ask_size2']
        )

        df["spread"] = df['ask_price1'] / df['bid_price1'] - 1

        d = pd.DataFrame(
            {
                "stock_id": df["stock_id"],
                "wap": df["wap"],
                "dom": df["dom"],
                "spread": df["spread"],
                "seconds": df["seconds_in_bucket"],
                "time_id":df['time_id']
            }
        )
        d = d.reset_index(drop=True)
        index_range = pd.Index(range(600), name='seconds')
        d = d.set_index('seconds').reindex(index_range)

        # Forward fill
        d = d.ffill().reset_index()
        
        # Back fill 
        d = d.bfill().iloc[:600]
        d = d.reset_index(drop=True)

        stocks = pd.concat([stocks, d]).astype({'stock_id': 'int', 'seconds': 'int', 'time_id': 'int'})

    stocks['wap_mean'] = stocks.groupby(['stock_id'])['wap'].transform('mean')

    stocks['market_mean_seconds'] = stocks.groupby(['seconds'])['wap'].transform('mean')

    market_mean = stocks['wap'].mean()
    stocks['market_mean'] = market_mean

    stocks = stocks.groupby(['stock_id']).apply(compute_beta)

    stocks['dom_mean'] = stocks.groupby(['stock_id'])['dom'].transform('mean')

    stocks['spread_mean'] = stocks.groupby(['stock_id'])['spread'].transform('mean')

    stock_betas = pd.DataFrame(
        {
            'stock_id': stocks['stock_id'].unique(),
            'beta': stocks['beta'].unique(),
            'dom': stocks['dom_mean'].unique(),
            'spread': stocks['spread_mean'].unique(),
        }
    )

    return stock_betas

files = glob.glob(os.path.join("./individual_book_train", "*.csv"))
time_ids = pd.read_csv(files[0]).time_id.unique()
stocks = pd.read_csv("all_times.csv")
stocks = stocks.loc[:, ~stocks.columns.str.contains('^Unnamed')]

perc = 0

for i, time_id in enumerate(sorted(time_ids, reverse=True)):
    if time_id in stocks.time_id.unique() or time_id in [2458, 3138, 5285, 6172, 6696, 6925, 32186, 29802]:
        continue
    
    print(time_id)

    s = extract_features(files, time_id)
    s['time_id'] = time_id
    
    #print(time_id)
    
    s.to_csv(f"time/time_{time_id}.csv")
    #print(time_id)
    
    stocks = pd.concat([stocks, s]).astype({'stock_id': 'int', 'time_id': 'int'})
    stocks = stocks.sort_values(by='stock_id').reset_index(drop=True)
    stocks = stocks.loc[:, ~stocks.columns.str.contains('^Unnamed')]

    stocks.to_csv("all_times.csv", index=False)
    
    if i % (len(time_ids)//10) == 0:
        perc += 10
        print(f"{perc}%")
        
#stocks.to_csv("all_times.csv", index=False)

29793
29791
29786
29775
29761
29746
29740
29716
29692
29679
29676
29672


In [ ]:
stocks

In [ ]:
stocks.to_csv("all_times.csv", index=False)

In [ ]:
stocks = pd.read_csv("all_times.csv")
stocks